In [ ]:
import os 

import cv2 
import matplotlib.pyplot as plt 
import numpy as np

# Эти библиотеки нужны для 3Д визуализации
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors

import ipywidgets as widgets
from ipywidgets import interact

## Загрузка данных 

Чтобы пощупать сегментацию на базе цветового пространства, мы будем использовать [маленький датасет с рыбами-клоунами](https://github.com/realpython/materials/tree/master/opencv-color-spaces). 

> Нужно скачать папку images, переименовать её в `clownfishes`, и положить её в папку `/notebook/CV/data`.  

Давайте загрузим первую картинку.

In [ ]:
fpath = os.path.join(os.path.dirname(os.path.abspath(os.getcwd())), "data", "clownfishes", "nemo0.jpg")
original_image = cv2.imread(fpath)
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=[8, 6])
plt.imshow(original_image)
plt.show()

## Визуализация Немо в RGB-пространстве 

Пространство `HSV` - это хороший выбор для сегментации цветов, но чтобы понять почему, давайте сначала сравним изображение в `RGB` с изображением в `HSV` пространстве при помощи визуализации распределения цветов. 

In [ ]:
r, g, b = cv2.split(original_image)

fig = plt.figure(figsize=[8, 6])
axis = fig.add_subplot(1, 1, 1, projection="3d")
pixel_colors = original_image.reshape((np.shape(original_image)[0]*np.shape(original_image)[1], 3))
norm = colors.Normalize(vmin=-1.,vmax=1.)
norm.autoscale(pixel_colors)
pixel_colors = norm(pixel_colors).tolist()
axis.scatter(r.flatten(), g.flatten(), b.flatten(), facecolors=pixel_colors, marker=".")
axis.set_xlabel("Red")
axis.set_ylabel("Green")
axis.set_zlabel("Blue")
plt.show()

Из этого графика видно, что оранжевая часть картинки размазана почти по всему диапазону красных, зелёных и синих значений. 

Поскольку тело рыбки Немо заполняет почти весь график, то сегментировать его тело на базе RGB пространства будет довольно сложно. 

## Визуализация Немо в HSV-пространстве 

В RGB на рыбку мы посмотрели, давайте получим такой же график, но в HSV. 

Давайте сконвертируем рыбку из RGB в HSV 

In [ ]:
hsv_nemo = cv2.cvtColor(original_image, cv2.COLOR_RGB2HSV)

plt.figure(figsize=[8, 6])
plt.imshow(hsv_nemo)
plt.title("HSV Image")
plt.show()

И получим такой же график распределения значений. 

In [ ]:
h, s, v = cv2.split(hsv_nemo)
fig = plt.figure(figsize=[8, 6])
axis = fig.add_subplot(1, 1, 1, projection="3d")

axis.scatter(h.flatten(), s.flatten(), v.flatten(), facecolors=pixel_colors, marker=".")
axis.set_xlabel("Hue")
axis.set_ylabel("Saturation")
axis.set_zlabel("Value")
plt.show()

В пространстве HSV оранжевый цвет рыбки намного более локализован и визуально его легко отделить от остальных цветов. 

Насыщенность и яркость оранжевого цвета действительно различаются, но всё же основное количество цвета расположено в небольшом диапазоне по оси тона (Hue). 

Это и является ключевым моментом, который можно использовать для сегментации рыбки.

## Создание масок сегментации

Давайте подберём пороговое значение для оранжевого цвета в Немо.

То есть наша задача - это найти такие пороговые значения для HSV, чтобы на картинке осталось только оранжевое тело рыбы. 

Попробуйте сначала проиграться с порогами. 

<details><summary>Если уж очень не хочется, то вот вам спойлер😜</summary>
    
```python
low_orange_color=(1, 190, 200)
high_orange_color=(18, 255, 255)
```

</details>

In [ ]:
@interact
def choose_threshold(
    low_hue=widgets.IntSlider(value=0, min=0, max=255), 
    low_sat=widgets.IntSlider(value=0, min=0, max=255), 
    low_val=widgets.IntSlider(value=0, min=0, max=255), 
    high_hue=widgets.IntSlider(value=255, min=0, max=255),
    high_sat=widgets.IntSlider(value=255, min=0, max=255),
    high_val=widgets.IntSlider(value=255, min=0, max=255), 
):
    low_orange_color = (low_hue, low_sat, low_val)
    high_orange_color = (high_hue, high_sat, high_val)

    orange_mask = cv2.inRange(hsv_nemo, low_orange_color, high_orange_color)
    result = cv2.bitwise_and(original_image, original_image, mask=orange_mask)

    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=[15, 10])
    ax[0].imshow(orange_mask, cmap="gray")
    ax[0].set_title("Orange Mask")

    ax[1].imshow(result)
    ax[1].set_title("Original Image with Mask")

    plt.show()


Теперь мы можем детектировать оранжевую часть тела Немо. Но есть проблема, у рыбки есть ещё и белые полосы. 

Всё просто, давайте сделаем вторую маску, которая детектирует белые полосы, по аналогии с оранжевым цветом. 

<details><summary>Спойлер для белого цвета😜</summary>
    
```python
low_white_color=(0, 0, 200)
high_white_color=(145, 60, 255)
```

</details>

In [ ]:
@interact
def choose_threshold(
    low_hue=widgets.IntSlider(value=0, min=0, max=255), 
    low_sat=widgets.IntSlider(value=0, min=0, max=255), 
    low_val=widgets.IntSlider(value=0, min=0, max=255), 
    high_hue=widgets.IntSlider(value=255, min=0, max=255),
    high_sat=widgets.IntSlider(value=255, min=0, max=255),
    high_val=widgets.IntSlider(value=255, min=0, max=255), 
):
    low_white_color = (low_hue, low_sat, low_val)
    high_white_color = (high_hue, high_sat, high_val)

    white_mask = cv2.inRange(hsv_nemo, low_white_color, high_white_color)
    result = cv2.bitwise_and(original_image, original_image, mask=white_mask)

    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=[15, 10])
    ax[0].imshow(white_mask, cmap="gray")
    ax[0].set_title("White Mask")

    ax[1].imshow(result)
    ax[1].set_title("Original Image with Mask")

    plt.show()

А теперь осталось только скомбинировать наши две маски. То есть там, где есть оранжевый или белый цвет, будет 1, а всё остальное 0. Давайте посмотрим, что получается. 

In [ ]:
# Сюда можете подставить свои значения для оранжевого цвета
low_orange_color=(1, 190, 200)
high_orange_color=(18, 255, 255)
orange_mask = cv2.inRange(hsv_nemo, low_orange_color, high_orange_color)

# Сюда можете подставить свои значения для белого цвета
low_white_color=(0, 0, 200)
high_white_color=(145, 60, 255)
white_mask = cv2.inRange(hsv_nemo, low_white_color, high_white_color)

final_mask = orange_mask + white_mask
final_result = cv2.bitwise_and(original_image, original_image, mask=final_mask)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=[15, 10])
ax[0].imshow(final_mask, cmap="gray")
ax[0].set_title("Mask")

ax[1].imshow(final_result)
ax[1].set_title("Original Image with Mask")

plt.show()

Та-дам! По сути у нас получилась грубая сегментация в цветовом пространстве HSV. 

Да, видно, что маска не идеальная и есть паразитные пиксели. Можно попробовать немного улучшить ситуацию с помощью размытия по Гауссу. Это поможет убрать некоторые ложные пиксели. 

Размытие по Гауссу - это фильтр, который использует функцию Гаусса для преобразования каждого пикселя на изображении. В результате такой фильтр сглаживает шумы на изображении и уменьшает детализацию. 

Давайте посмотрим, что случится с рыбкой, если натравить на неё Гаусса. 

In [ ]:
blur = cv2.GaussianBlur(final_result, (7, 7), 0)

plt.figure(figsize=[8, 6])
plt.imshow(blur)
plt.show()

## Оценка работы сегментации

Ради интереса, давайте посмотрим, насколько хорошо наш метод сегментации работает на других картинках с рыбами-клоунами. 

Сначала давайте загрузим все картинки.

In [ ]:
dpath = os.path.join(os.path.dirname(os.path.abspath(os.getcwd())), "data", "clownfishes")

all_images = []
for file_name in os.listdir(dpath):
    img = cv2.cvtColor(
        cv2.imread(os.path.join(dpath, file_name)), 
        cv2.COLOR_BGR2RGB
    )
    all_images.append(img)

print(f"Найдено {len(all_images)} картинок.")

Давайте создадим отдельную функцию, в которую положим все наши предыдущие наработки. 

In [ ]:
def segment_clownfish(image: np.ndarray):
    # Конвертируем картинку в HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    # Задаём пороги для оранжевого цвета
    light_orange = (1, 190, 200)
    dark_orange = (18, 255, 255)
    # Применяем маску для оранжевого цвета
    mask = cv2.inRange(hsv_image, light_orange, dark_orange)

    # Задаём пороги для белого цвета
    light_white = (0, 0, 200)
    dark_white = (145, 60, 255)
    # Применяем маску для белого цвета
    mask_white = cv2.inRange(hsv_image, light_white, dark_white)
    
    # Соединяем две маски
    final_mask = mask + mask_white
    result = cv2.bitwise_and(image, image, mask=final_mask)

    # Применяем размытие Гаусса для размытия шумов
    blur = cv2.GaussianBlur(result, (7, 7), 0)
    return blur

Теперь, когда у нас есть функция, мы можем сегментировать все наши картинки, буквально за одну строчку кода. 

In [ ]:
results = [segment_clownfish(friend) for friend in all_images]

In [ ]:
for i in range(1, 6):
    plt.subplot(1, 2, 1)
    plt.imshow(all_images[i])
    plt.subplot(1, 2, 2)
    plt.imshow(results[i])
    plt.show()

## Заключение 

В целом с помощью такого простого метода сегментации можно найти большинство рыбок. Однако очевидно, что у такого подхода есть свои недостатки, так например, при малейшем изменении освещения или при появлении теней, наши пороговые значения уже не будут работать и найти рыбку не получится.

## Полезные ссылки 

* [RealPython статья](https://realpython.com/python-opencv-color-spaces/)